In [7]:
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
from bs4 import BeautifulSoup
from collections import defaultdict

In [8]:
folder = 'data/TAC2017/'

In [9]:
train_labels = glob(folder+'train_xml/*')

drug2mentions = defaultdict(set)
drug2reactions = defaultdict(set)

for label in tqdm(train_labels):
    drug_name = label.split('/')[-1].split('.')[0]
    with open(label, 'r') as f:
        soup = BeautifulSoup(f, 'xml')
    
    for mention in soup.find_all('Mention'):
        if mention['type'] == 'AdverseReaction':
          section_name = mention['section']
          mention_str = mention['str'].lower()
          drug2mentions[drug_name].add(mention_str)
    
    for reaction in soup.find_all('Reaction'):
        reaction_str = reaction['str']
        # for norm in reaction.find_all('Normalization'):
        #   try:
        #     meddra_str = norm['meddra_pt']
        #     meddra_id = norm['meddra_pt_id']
        #   except:
        #     meddra_str, meddra_id = None, None
        #   try:
        #     meddra_llt = norm['meddra_llt']
        #     meddra_llt_id = norm['meddra_llt_id']
        #   except:
        #     meddra_llt, meddra_llt_id = None, None
        # drug_label_text.append([drug_name, string, meddra_str, meddra_id, meddra_llt, meddra_llt_id])
        drug2reactions[drug_name].add(reaction_str)

len(drug2mentions), len(drug2reactions)

100%|██████████| 101/101 [00:00<00:00, 219.88it/s]


(101, 101)

In [12]:
diffs_list = list()

for drug in drug2mentions.keys():

    setdiff = len(drug2mentions[drug]-drug2reactions[drug])
    diff = len(drug2mentions[drug])-len(drug2reactions[drug])
    diffs_list.append([drug, len(drug2mentions[drug]), len(drug2reactions[drug]), diff, setdiff])

XEOMIN 81 72 9
STRIBILD 136 135 1
PRISTIQ 148 128 20
TAFINLAR 158 155 3
MULTAQ 57 56 1
TANZEUM 60 54 6
BELEODAQ 51 50 1
HALAVEN 66 65 1
XIAFLEX 95 94 1
CIMZIA 172 146 26
GRANIX 35 26 9
JEVTANA 80 77 3
ILARIS 61 59 2
ARCAPTA 53 38 15
POTIGA 108 107 1
BREO 87 72 15
NESINA 25 24 1
EYLEA 30 29 1
TECFIDERA 41 38 3
TREANDA 154 149 5
BOSULIF 105 104 1
GILENYA 130 128 2
XALKORI 99 96 3
CLEVIPREX 19 17 2
PROLIA 92 91 1
FANAPT 214 196 18
ONFI 63 62 1
COMETRIQ 111 105 6
NULOJIX 118 116 2
GILOTRIF 80 79 1
APTIOM 102 95 7
DUAVEE 92 80 12
TIVICAY 71 62 9
TRULICITY 55 50 5
TEFLARO 38 36 2
JARDIANCE 69 68 1
YERVOY 96 92 4
GADAVIST 63 62 1
OTEZLA 35 34 1
FIRAZYR 11 9 2
XTANDI 70 68 2
AFINITOR 261 259 2
FERRIPROX 97 93 4
PRADAXA 72 70 2
HORIZANT 62 57 5
STENDRA 72 62 10
ADCETRIS 91 88 3
ZERBAXA 52 51 1
ZYDELIG 73 72 1
SIMPONI 128 115 13
NATAZIA 101 74 27
EDARBI 48 37 11
IMBRUVICA 89 88 1
DUREZOL 58 37 21
ZYKADIA 92 91 1
ERWINAZE 41 38 3
SAPHRIS 216 187 29
INLYTA 116 113 3
